# 🔍 SAP ABAP SY-UNAME 추적기: 변수 추적 원리 이해하기

## 📚 학습 목표
1. **Taint Analysis (오염 분석)** 개념 이해
2. **변수 전파 (Variable Propagation)** 원리 학습
3. **정규식 패턴 매칭** 기법 체험
4. **Sink Detection (종착점 감지)** 메커니즘 이해
5. **실제 ABAP 코드**에서 추적 과정 시뮬레이션

---

## 🎯 핵심 개념: Taint Analysis

**Taint Analysis**는 보안 분야에서 널리 사용되는 정적 분석 기법입니다:

```
Source (오염원) → Propagation (전파) → Sink (종착점)
   SY-UNAME   →   변수 할당들    →  DB/RFC 호출
```

### 🔬 기본 원리
1. **특정 변수(SY-UNAME)를 "오염된" 상태로 표시**
2. **오염된 변수가 다른 변수에 할당되면 그 변수도 오염됨**
3. **오염된 변수가 중요한 곳(DB, RFC)에 사용되면 추적 완료**


In [ ]:
# 필요한 라이브러리 import
import re
import json
from typing import Set, List, Dict

print("🚀 SAP ABAP SY-UNAME 추적기 학습 시작!")
print("="*50)


## 📖 Chapter 1: Tainted Variables (오염된 변수) 개념

### 🧪 실습 1: 기본 Taint Tracking


In [ ]:
# 간단한 Taint Tracker 클래스 구현
class SimpleTaintTracker:
    def __init__(self):
        self.tainted_vars = set()  # 오염된 변수들을 저장
        self.trace_path = []       # 추적 경로를 저장
    
    def mark_tainted(self, variable):
        """변수를 오염된 상태로 표시"""
        self.tainted_vars.add(variable.lower())
        print(f"🦠 '{variable}' 변수가 오염되었습니다!")
    
    def is_tainted(self, variable):
        """변수가 오염되었는지 확인"""
        return variable.lower() in self.tainted_vars
    
    def propagate(self, source, target, line_num):
        """오염 전파: source가 오염되어 있으면 target도 오염"""
        if self.is_tainted(source):
            self.tainted_vars.add(target.lower())
            step = f"Line {line_num}: '{source}' → '{target}'"
            self.trace_path.append(step)
            print(f"🔗 오염 전파: {step}")
            return True
        return False
    
    def show_status(self):
        """현재 상태 출력"""
        print(f"\n📊 현재 오염된 변수들: {list(self.tainted_vars)}")
        print(f"📈 추적 경로: {len(self.trace_path)}단계")
        for i, step in enumerate(self.trace_path, 1):
            print(f"  {i}. {step}")

# 테스트해보기
tracker = SimpleTaintTracker()

# 1. SY-UNAME을 오염 소스로 설정
tracker.mark_tainted("sy-uname")
tracker.show_status()


In [ ]:
# 실제 ABAP 코드 시뮬레이션
print("\n🎭 ABAP 코드 시뮬레이션:")
print("Line 90: lv_lifnr = sy-uname.")
print("Line 104: SELECT ... FROM ekko WHERE lifnr = @lv_lifnr.")
print()

# 2. 변수 할당 추적
tracker.propagate("sy-uname", "lv_lifnr", 90)

# 3. 현재 상태 확인
tracker.show_status()

# 4. Sink 감지 (SELECT WHERE 절에서 사용)
if tracker.is_tainted("lv_lifnr"):
    print("\n🎯 Sink 감지! lv_lifnr이 SELECT WHERE 절에 사용됨")
    print("   → 테이블: EKKO, 필드: LIFNR")
    print("   → SY-UNAME이 데이터베이스 조회에 사용되는 것을 확인!")


## 📖 Chapter 2: 정규식 패턴 매칭의 마법

### 🔍 실습 2: ABAP 패턴 인식하기

여러분의 프로젝트는 **21가지 ABAP 패턴**을 정규식으로 식별합니다. 이것이 바로 핵심 기술입니다!


In [ ]:
# 실제 프로젝트에서 사용하는 주요 정규식 패턴들
class ABAPPatternMatcher:
    def __init__(self):
        # 기본 할당 패턴: target = source.
        self.assign_pattern = re.compile(
            r"^\s*(?P<target>[\w\d\-\>\[\]]+)\s*=\s*(?P<source>[\w\d\-\>\[\]]+)\s*\.",
            re.IGNORECASE
        )
        
        # RFC 호출 패턴: CALL FUNCTION 'RFC_NAME'
        self.rfc_pattern = re.compile(
            r"^\s*CALL\s+FUNCTION\s+'(?P<rfc_name>[\w\d_]+)'\s*(?P<params>.*)",
            re.IGNORECASE | re.DOTALL
        )
        
        # SELECT WHERE 패턴
        self.select_where_pattern = re.compile(
            r"^\s*SELECT.*?FROM\s+(?P<table>[\w\d_]+).*?WHERE.*?(?P<field>[\w\d_]+)\s*=\s*@?(?P<variable>[\w\d\-\>\[\]]+)",
            re.IGNORECASE | re.DOTALL
        )
        
        # INSERT 패턴
        self.insert_pattern = re.compile(
            r"^\s*INSERT\s+(?:INTO\s+)?(?P<table>[\w\d_]+)\s+(?:VALUES\s+(?P<values>.*?)|FROM\s+(?P<source>[\w\d\-\>\[\]]+))\s*\.",
            re.IGNORECASE | re.DOTALL
        )
    
    def analyze_line(self, line):
        """ABAP 라인을 분석하여 패턴 매칭"""
        line_upper = line.strip().upper()
        results = []
        
        # 1. 할당 패턴 체크
        assign_match = self.assign_pattern.match(line_upper)
        if assign_match:
            results.append({
                'type': 'ASSIGNMENT',
                'source': assign_match.group('source').lower(),
                'target': assign_match.group('target').lower()
            })
        
        # 2. RFC 패턴 체크
        rfc_match = self.rfc_pattern.match(line_upper)
        if rfc_match:
            results.append({
                'type': 'RFC_CALL',
                'rfc_name': rfc_match.group('rfc_name'),
                'params': rfc_match.group('params')
            })
        
        # 3. SELECT WHERE 패턴 체크
        select_match = self.select_where_pattern.match(line_upper)
        if select_match:
            results.append({
                'type': 'SELECT_WHERE',
                'table': select_match.group('table'),
                'field': select_match.group('field'),
                'variable': select_match.group('variable').lower()
            })
        
        # 4. INSERT 패턴 체크
        insert_match = self.insert_pattern.match(line_upper)
        if insert_match:
            results.append({
                'type': 'INSERT',
                'table': insert_match.group('table'),
                'source': insert_match.group('source')
            })
        
        return results

# 패턴 매처 생성
matcher = ABAPPatternMatcher()
print("🎯 ABAP 패턴 매처 준비 완료!")


In [ ]:
# 실제 ABAP 코드 라인들로 테스트
test_lines = [
    "  lv_lifnr = sy-uname.",
    "  lv_current_user = sy-uname.",
    "  ls_document-created_by = lv_current_user.",
    "  INSERT zdocuments FROM ls_document.",
    "  SELECT * FROM ekko INTO lt_ekko WHERE lifnr = @lv_lifnr.",
    "  CALL FUNCTION 'BAPI_USER_GET_DETAIL' EXPORTING username = lv_user_id."
]

print("🔬 ABAP 코드 라인 분석 결과:\n")

for i, line in enumerate(test_lines, 1):
    print(f"📝 Line {i}: {line}")
    
    patterns = matcher.analyze_line(line)
    
    if patterns:
        for pattern in patterns:
            if pattern['type'] == 'ASSIGNMENT':
                print(f"   ✅ 할당 감지: {pattern['source']} → {pattern['target']}")
            elif pattern['type'] == 'RFC_CALL':
                print(f"   ✅ RFC 호출: {pattern['rfc_name']}")
            elif pattern['type'] == 'SELECT_WHERE':
                print(f"   ✅ SELECT WHERE: 테이블={pattern['table']}, 필드={pattern['field']}, 변수={pattern['variable']}")
            elif pattern['type'] == 'INSERT':
                print(f"   ✅ INSERT: 테이블={pattern['table']}, 소스={pattern['source']}")
    else:
        print("   ❌ 매칭되는 패턴 없음")
    
    print()


## 📖 Chapter 3: 전체 추적 시스템 통합

### 🎪 실습 3: 실제 프로젝트 로직 체험

이제 패턴 매칭과 Taint Analysis를 결합해서 실제 프로젝트의 핵심 로직을 구현해보겠습니다!


In [ ]:
# 실제 프로젝트의 핵심 로직을 단순화한 버전
class ABAPTaintAnalyzer:
    def __init__(self):
        self.tainted_vars = {"sy-uname"}  # SY-UNAME으로 시작
        self.trace_path = []
        self.matcher = ABAPPatternMatcher()
    
    def analyze_snippet(self, lines):
        """코드 스니펫 전체를 분석"""
        print("🔍 코드 스니펫 분석 시작...")
        print(f"📊 초기 오염된 변수: {list(self.tainted_vars)}\n")
        
        for line_num, line in enumerate(lines, 1):
            print(f"📝 Line {line_num}: {line.strip()}")
            
            # 패턴 매칭
            patterns = self.matcher.analyze_line(line)
            
            for pattern in patterns:
                if pattern['type'] == 'ASSIGNMENT':
                    self._handle_assignment(pattern, line_num)
                elif pattern['type'] == 'SELECT_WHERE':
                    result = self._handle_select_where(pattern, line_num)
                    if result:
                        return result
                elif pattern['type'] == 'INSERT':
                    result = self._handle_insert(pattern, line_num)
                    if result:
                        return result
                elif pattern['type'] == 'RFC_CALL':
                    result = self._handle_rfc_call(pattern, line_num)
                    if result:
                        return result
            print()
        
        return {"status": "Not Found", "message": "중요한 Sink를 찾지 못했습니다"}
    
    def _handle_assignment(self, pattern, line_num):
        """할당 패턴 처리"""
        source = pattern['source']
        target = pattern['target']
        
        if source in self.tainted_vars:
            self.tainted_vars.add(target)
            step = f"Line {line_num}: Assignment '{source}' → '{target}'"
            self.trace_path.append(step)
            print(f"   🔗 오염 전파: {step}")
            print(f"   📊 현재 오염된 변수: {list(self.tainted_vars)}")
    
    def _handle_select_where(self, pattern, line_num):
        """SELECT WHERE 패턴 처리"""
        variable = pattern['variable']
        
        if variable in self.tainted_vars:
            print(f"   🎯 SELECT WHERE Sink 감지!")
            print(f"   📊 테이블: {pattern['table']}, 필드: {pattern['field']}")
            
            return {
                "status": "Found",
                "type": "DATABASE_SELECT_WHERE",
                "table": pattern['table'],
                "field": pattern['field'],
                "variable": variable,
                "trace_path": self.trace_path.copy(),
                "tainted_vars": list(self.tainted_vars)
            }
        
        return None
    
    def _handle_insert(self, pattern, line_num):
        """INSERT 패턴 처리"""
        source = pattern['source']
        
        if source and source.lower() in self.tainted_vars:
            print(f"   🎯 INSERT Sink 감지!")
            print(f"   📊 테이블: {pattern['table']}")
            
            return {
                "status": "Found",
                "type": "DATABASE_INSERT",
                "table": pattern['table'],
                "source": source,
                "trace_path": self.trace_path.copy(),
                "tainted_vars": list(self.tainted_vars)
            }
        
        return None
    
    def _handle_rfc_call(self, pattern, line_num):
        """RFC 호출 패턴 처리 (간단화)"""
        print(f"   🎯 RFC 호출 감지!")
        print(f"   📞 RFC 이름: {pattern['rfc_name']}")
        
        return {
            "status": "Found",
            "type": "RFC",
            "rfc_name": pattern['rfc_name'],
            "trace_path": self.trace_path.copy(),
            "tainted_vars": list(self.tainted_vars)
        }

print("🚀 ABAP Taint Analyzer 준비 완료!")


In [ ]:
# 실제 시나리오 1: SELECT WHERE 추적
print("🎭 시나리오 1: SY-UNAME → SELECT WHERE 추적")
print("="*60)

scenario1_code = [
    "  lv_lifnr = sy-uname.",
    "  SELECT * FROM ekko INTO lt_ekko WHERE lifnr = @lv_lifnr."
]

analyzer1 = ABAPTaintAnalyzer()
result1 = analyzer1.analyze_snippet(scenario1_code)

print("\n🎯 분석 결과:")
print(json.dumps(result1, indent=2, ensure_ascii=False))
